# Tour Design

## Assumption:
- Assuming there is traveling cost between venues noted as $C_{ij}$

## Inputs:
- $C_{ij}$: Traveling cost from venue $i$ to venue $j$
- $R_i$: Expected revenue of venue $i$
- $V_k$: Number of selected venues in city $k$
- $i, j$: index of venues, $i, j = 1 ... N$
- $k$: index of cities, $k\in \mathbb{N}^+$
- $N$: Total number of venues
an empty city

## Decision:
- $x_{ij}$: Whether to go from venue $i$ to venue $j$
- $y_i$: Whether venue $i$ is selected
- $u_i$: order number for venue $i$ 
## Objectives:
$$
\text{Maximize } \sum_{i=1}^{N} R_i y_i - \sum_{i=1}^{N} \sum_{j=1}^{N} C_{ij} x_{ij}
$$

## Constraints:
- $\sum_{i=1}^{N} y_i = 25$. Total 25 stops in the tour route.
- $x_{ij}, y_{ij} \in \{0, 1\}$. Namely $x_{ij}, y_{ij}$ are binary.
- $\sum_{i \in V_k} y_{i} \le 2$. Each city has maximum 2 venue selected.
- $\sum_{i \in V_{city}} y_i \ge 1, V_{city} \in \{Portland, Miami, Phoenix\}$. At lease one stop in mandatory cities.
- $\sum_{j=1}^{N} x_{ij} = y_i$. Only one outward flow from city $j$.
- $\sum_{i=1}^{N} x_{ij} = y_j$. Only one inward flow towards city $j$.
- $x_{ii}=0$ no self loop. 
- $u_i - u_j + 25 * x_{ij} <= 24$ Sub-tour elimination constraints (Miller–Tucker–Zemlin)

In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import random
from itertools import combinations

#********start of user block********************

# Parameters
cities = ['Portland', 'Miami', 'Phoenix', 'Los Angeles', 'FakeCity']
mandatory_cities = ['Miami','FakeCity']  # Cities that must be included in the solution
num_venues_per_city = 3  # Default number of venues per city
max_venues_num=2  #max venues visited per city
total_venues=6   #total venues visited

#********end of user block********************

max_venues = {city: max_venues_num for city in cities if city != 'FakeCity'}
max_venues['FakeCity'] = 1  # FakeCity has only one venue

venues = {city: range(num_venues_per_city) for city in cities if city != 'FakeCity'}
venues['FakeCity'] = range(1)  # FakeCity has only one venue

# Set random seed for reproducibility
random.seed(42)

# Expected revenues (randomly generated for simplicity)
revenues = {city: [random.randint(50, 200) for _ in venues[city]] for city in cities}
revenues['FakeCity'] = [0]

# Traveling costs (randomly generated for simplicity)
costs = {(city1, venue1, city2, venue2): random.randint(10, 100) if city1 != 'FakeCity' and city2 != 'FakeCity' else 0
         for city1 in cities for venue1 in venues[city1]
         for city2 in cities for venue2 in venues[city2]}

# Create the model
model = Model('Maximize_Revenue')

# Decision variables
x = model.addVars([(city1, venue1, city2, venue2) for city1 in cities for venue1 in venues[city1]
                                                             for city2 in cities for venue2 in venues[city2]], vtype=GRB.BINARY, name='x')
y = model.addVars([(city, venue) for city in cities for venue in venues[city]], vtype=GRB.BINARY, name='y')
u = model.addVars([(city, venue) for city in cities for venue in venues[city]], vtype=GRB.INTEGER, name='u')

# Objective function
model.setObjective(
    quicksum(revenues[city][venue] * y[city, venue] for city in cities for venue in venues[city]) -
    quicksum(costs[(city1, venue1, city2, venue2)] * x[city1, venue1, city2, venue2] for city1 in cities for venue1 in venues[city1]
                                                                                     for city2 in cities for venue2 in venues[city2]),
    GRB.MAXIMIZE
)

# Constraints
# Total venues constraint
model.addConstr(quicksum(y[city, venue] for city in cities for venue in venues[city]) == total_venues + 1, name="TotalVenues")


# Maximum venues per city constraint
for city in cities:
    model.addConstr(quicksum(y[city, venue] for venue in venues[city]) <= max_venues[city], name=f"MaxVenues_{city}")

# Mandatory cities constraint
for city in mandatory_cities:
    model.addConstr(quicksum(y[city, venue] for venue in venues[city]) >= 1, name=f"Mandatory_{city}")

    
# Define y in terms of x
for city1 in cities:
    for venue1 in venues[city1]:
        model.addConstr(quicksum(x[city1, venue1, city2, venue2]
                                 for city2 in cities
                                 for venue2 in venues[city2]
                                ) == y[city1, venue1], name=f"Out_{city1}_{venue1}")
        model.addConstr(quicksum(x[city2, venue2, city1, venue1]
                                 for city2 in cities
                                 for venue2 in venues[city2]
                                ) == y[city1, venue1], name=f"In_{city1}_{venue1}")


# MTZ Constraints to eliminate subtours
for city1 in cities:
    for venue1 in venues[city1]:
        for city2 in cities:
            for venue2 in venues[city2]:
                if (city1, venue1) != (city2, venue2):
                    model.addConstr(u[city1, venue1] - u[city2, venue2] + (total_venues+1) * x[city1, venue1, city2, venue2] <= total_venues, name=f"MTZ_{city1}_{venue1}_{city2}_{venue2}")

# No self-loop
for city in cities:
    for venue in venues[city]:
        model.addConstr(x[city, venue, city, venue] == 0, name=f"No_selfLoop_{city}_{venue}")
        
# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Total Expected Revenue: {model.objVal}")
    for city in cities:
        for venue in venues[city]:
            if y[city, venue].X > 0.5:
                print(f"Visit venue {venue} in {city} with expected revenue {revenues[city][venue]}")
else:
    print("No optimal solution found")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 203 rows, 195 columns and 875 nonzeros
Model fingerprint: 0xe10fe673
Variable types: 0 continuous, 195 integer (182 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 15 rows and 14 columns
Presolve time: 0.00s
Presolved: 188 rows, 181 columns, 831 nonzeros
Variable types: 0 continuous, 181 integer (168 binary)

Root relaxation: objective 6.415000e+02, 55 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

In [2]:
# Print results
if model.status == GRB.OPTIMAL:
    print(f"Total Expected Revenue: {model.objVal}")
                
    # Print x variables
    for city1 in cities:
        for venue1 in venues[city1]:
            for city2 in cities:
                for venue2 in venues[city2]:
                    if x[city1, venue1, city2, venue2].x > 0.5:
                        print(f"x[{city1}, {venue1}, {city2}, {venue2}] = {x[city1, venue1, city2, venue2].x}")
                        print(f"y[{city1}, {venue1}] = {y[city1, venue1].x}")
                        print(f"y[{city2}, {venue2}] = {y[city2, venue2].x}")
    
else:
    print("No optimal solution found")

No optimal solution found


In [3]:
all_venu=[]
for city1 in cities:
        for venue1 in venues[city1]:
            for city2 in cities:
                for venue2 in venues[city2]:
                    if x[city1, venue1, city2, venue2].x > 0.5:
                        all_venu.append([city1, venue1, city2, venue2])
prev=('FakeCity', 0)
count=0
print(f'Stop {count}: {prev}')
fackcity_count=0
while True:
    for [city1, venue1, city2, venue2] in all_venu:
        if (city1, venue1) == prev:
            print()
            prev=(city2, venue2)
            count+=1
            print(f'Stop {count}: {prev}')
            #print([city1, venue1, city2, venue2])
            break
    if prev==('FakeCity', 0):
        break


AttributeError: Unable to retrieve attribute 'x'

In [4]:
# Print results
if model.status == GRB.OPTIMAL:
    print(f"Total Expected Revenue: {model.objVal}")
                
    # Print x variables
    for city1 in cities:
        for venue1 in venues[city1]:
            for city2 in cities:
                for venue2 in venues[city2]:
                    
                    print(f"x[{city1}, {venue1}, {city2}, {venue2}] = {x[city1, venue1, city2, venue2].x}")
                    print(f"y[{city1}, {venue1}] = {y[city1, venue1].x}")
                    print(f"y[{city2}, {venue2}] = {y[city2, venue2].x}")
    
else:
    print("No optimal solution found")

Total Expected Revenue: 450.0
x[Portland, 0, Portland, 0] = 0.0
y[Portland, 0] = 1.0
y[Portland, 0] = 1.0
x[Portland, 0, Portland, 1] = 0.0
y[Portland, 0] = 1.0
y[Portland, 1] = -0.0
x[Portland, 0, Miami, 0] = 1.0
y[Portland, 0] = 1.0
y[Miami, 0] = 1.0
x[Portland, 0, Miami, 1] = -0.0
y[Portland, 0] = 1.0
y[Miami, 1] = 0.0
x[Portland, 0, Phoenix, 0] = -0.0
y[Portland, 0] = 1.0
y[Phoenix, 0] = 1.0
x[Portland, 0, Phoenix, 1] = -0.0
y[Portland, 0] = 1.0
y[Phoenix, 1] = -0.0
x[Portland, 0, Los Angeles, 0] = -0.0
y[Portland, 0] = 1.0
y[Los Angeles, 0] = -0.0
x[Portland, 0, Los Angeles, 1] = -0.0
y[Portland, 0] = 1.0
y[Los Angeles, 1] = 1.0
x[Portland, 0, FakeCity, 0] = 0.0
y[Portland, 0] = 1.0
y[FakeCity, 0] = 1.0
x[Portland, 1, Portland, 0] = 0.0
y[Portland, 1] = -0.0
y[Portland, 0] = 1.0
x[Portland, 1, Portland, 1] = 0.0
y[Portland, 1] = -0.0
y[Portland, 1] = -0.0
x[Portland, 1, Miami, 0] = -0.0
y[Portland, 1] = -0.0
y[Miami, 0] = 1.0
x[Portland, 1, Miami, 1] = -0.0
y[Portland, 1] = -0.0
y